In [1]:

from mhr.utils.utils import load_json_file, process_jsonl,write_jsonl
from py2neo import *
from neo4j import GraphDatabase
from tqdm import tqdm


mme_result_file="/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/llava/model_args_ca10f3/0316_0009_mme_llava_baseline/mme_mul_en.json"
mme_token_file = "/mnt/petrelfs/songmingyang/songmingyang/data/llava_train/benchmarks/analysis/mme/mme_tokens.jsonl"
mme_output_file = "/mnt/petrelfs/songmingyang/songmingyang/data/llava_train/benchmarks/analysis/mme/mme_token_rank.jsonl"

# mme_object_file="/mnt/petrelfs/songmingyang/songmingyang/data/llava_train/benchmarks/analysis/mme/mme_llama_objects.jsonl"
# mme_output_file="/mnt/petrelfs/songmingyang/songmingyang/data/llava_train/benchmarks/analysis/mme/mme_co_occurrence_rank.jsonl"
# pope_result_file = "/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/llava/robustlmm/dr_algo/0601_2205_eval_all_llava_baseline/pope.json"
# pope_result_file = "/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/llava/model_args_ca10f3/0312_1719_eval_pope_mul_baseline/pope_mul_en.json"
# pope_object_file = "/mnt/petrelfs/songmingyang/songmingyang/data/llava_train/benchmarks/analysis/pope/pope_llama_objects.jsonl"
# pope_token_file = "/mnt/petrelfs/songmingyang/songmingyang/data/llava_train/benchmarks/analysis/pope/pope_tokens.jsonl"



# pope_output_file = "/mnt/petrelfs/songmingyang/songmingyang/data/llava_train/benchmarks/analysis/pope/pope_token_rank.jsonl"

# pope_aokvqa_file = "/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/llava/model_args_ca10f3/0321_1558_eval_pope_aokvqa_mul_baseline/pope_aokvqa_mul_en.json"
# pope_coco_file = "/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/llava/model_args_ca10f3/0321_1558_eval_pope_mul_baseline/pope_mul_en.json"
# pope_gqa_file = "/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/llava/model_args_ca10f3/0321_1559_eval_pope_gqa_mul_baseline/pope_gqa_mul_en.json"
# pope_result_file = [pope_aokvqa_file, pope_coco_file, pope_gqa_file]

In [2]:
def get_results_from_result_file(file):
    if isinstance(file, str):
        result = load_json_file(file)["logs"]
    elif isinstance(file, list):
        result = []
        for f in file:
            result.extend(load_json_file(f)["logs"])
    else:
        raise ValueError("file should be a string or a list of string")
    return result



def from_str_to_list(string):
    return string.strip().replace("[", "").replace("]", "").split(",")

def get_object_dict(object_file):
    object_data = process_jsonl(object_file)
    object_dict = {}
    for obj in object_data:
        object_dict[obj["id"]] = list(set(from_str_to_list(obj["objects"])))
    return object_dict

def get_token_dict(token_file):
    token_data = process_jsonl(token_file)
    token_dict = {}
    for token in token_data:
        token_dict[token["id"]] = token["tokens"]
    return token_dict
    

def get_object_statistic_by_neo4j(database="instructmix665kllama"):
    uri = "neo4j://10.140.54.16:7687"
    driver = GraphDatabase.driver(uri, auth=("neo4j", "12345678"))
    query = "match (n) return n;"
    with driver.session(database=database) as session:
        results = session.run(query).data()
        results.sort(key=lambda x: x['n']['count'], reverse=True)
        
    object_statistic_dict = {}
    for item in results:
        object_statistic_dict[item['n']['lemma']] = dict(num=item['n']['count'],rank=len(object_statistic_dict)+1)
    return object_statistic_dict,len(results)

def get_co_occurrence_by_neo4j(database="instructmix665kllama"):
    uri = "neo4j://10.140.54.16:7687"
    driver = GraphDatabase.driver(uri, auth=("neo4j", "12345678"))
    query = "match (a)-[r:CO_OCCURRENCE]->(b) return a.lemma, r.count, b.lemma;"
    with driver.session(database=database) as session:
        results = session.run(query).data()
        results.sort(key=lambda x: x['r.count'], reverse=True)
        
    object_statistic_dict = {}
    for item in results:
        object_statistic_dict[f"{item['a.lemma']},{item['b.lemma']}"] = dict(num=item['r.count'],rank=len(object_statistic_dict)+1)
    return object_statistic_dict,len(results)

def get_token_by_neo4j(database="instructmix665kpos;"):
    uri = "neo4j://10.140.54.16:7687"
    driver = GraphDatabase.driver(uri, auth=("neo4j", "12345678"))
    query = "match (n) return n;"
    with driver.session(database=database) as session:
        results = session.run(query).data()
        results.sort(key=lambda x: x['n']['count'], reverse=True)
        
    object_statistic_dict = {}
    for item in results:
        object_statistic_dict[item['n']['lemma']] = dict(num=item['n']['count'],rank=len(object_statistic_dict)+1)
    return object_statistic_dict,len(results)

def get_co_occurence_from_list(object_list):
    res = []
    object_list_dedup = list(set(object_list))
    for i in range(len(object_list_dedup)):
        for j in range(i+1,len(object_list_dedup)):
            if object_list_dedup[i] > object_list_dedup[j]:
                res.append(f"{object_list_dedup[j]},{object_list_dedup[i]}")
            else:
                res.append(f"{object_list_dedup[i]},{object_list_dedup[j]}")
    return res



In [4]:
# object_dict = get_object_dict(mme_token_file)
token_dict = get_token_dict(mme_token_file)
print(token_dict)
object_statistic_dict,total_instances = get_token_by_neo4j("instructmix665kpos")
# co_occurrence_dict,total_instances = get_co_occurrence_by_neo4j("instructmix665kllama")
result_list = get_results_from_result_file(mme_result_file)
analyze_results = []

wrong_max_ranks = []
correct_max_ranks = []
wrong_mean_ranks = []
correct_mean_ranks = []
wrong_min_ranks = []
correct_min_ranks = []

wrong_max_counts = []
correct_max_counts = []
wrong_mean_counts = []
correct_mean_counts = []
wrong_min_counts = []
correct_min_counts = []

for log in tqdm(result_list):
    id = log["doc"]["question_id"]
    image_key = log["doc"]["image"]
   
    score = log.get("mme_percetion_score", log.get("mme_cognition_score", None))
    # score = log["pope_accuracy"]
    if score is None:
        raise ValueError("score is None")
    score = score["score"]
    correct = score > 0.0
    
    now_token_list = token_dict.get(image_key,None)
    
    if now_token_list is None or len(now_token_list) == 0:
        continue
    current_num = []
    current_rank = []
    for object in now_token_list:
        if object_statistic_dict.get(object, None) is None:
            continue
        current_num.append(object_statistic_dict[object]["num"])
        current_rank.append(object_statistic_dict[object]["rank"])
    if len(current_num) == 0:
        # print("no co-occurrence")
        continue
    max_rank = max(current_rank)
    min_rank = min(current_rank)
    max_num = max(current_num)
    min_num = min(current_num)
    mean_rank = sum(current_rank) / len(current_rank)
    mean_num = sum(current_num) / len(current_num)
    if correct:
        correct_max_ranks.append(max_rank)
        correct_min_ranks.append(min_rank)
        correct_mean_ranks.append(mean_rank)
        correct_max_counts.append(max_num)
        correct_min_counts.append(min_num)
        correct_mean_counts.append(mean_num)
    else:
        wrong_max_ranks.append(max_rank)
        wrong_min_ranks.append(min_rank)
        wrong_mean_ranks.append(mean_rank)
        wrong_max_counts.append(max_num)
        wrong_min_counts.append(min_num)
        wrong_mean_counts.append(mean_num)
    
    
    analyze_results.append(dict(id=id, correct=correct, 
                                max_rank=max_rank, min_rank=min_rank, mean_rank=mean_rank, max_num=max_num, min_num=min_num, mean_num=mean_num,
                                rank_list=current_rank, num_list=current_num))
print("correct max rank:", sum(correct_max_ranks) / len(correct_max_ranks))
print("correct min rank:", sum(correct_min_ranks) / len(correct_min_ranks))
print("correct mean rank:", sum(correct_mean_ranks) / len(correct_mean_ranks))
print("wrong max rank:", sum(wrong_max_ranks) / len(wrong_max_ranks))
print("wrong min rank:", sum(wrong_min_ranks) / len(wrong_min_ranks))
print("wrong mean rank:", sum(wrong_mean_ranks) / len(wrong_mean_ranks))
print("correct max count:", sum(correct_max_counts) / len(correct_max_counts))
print("correct min count:", sum(correct_min_counts) / len(correct_min_counts))
print("correct mean count:", sum(correct_mean_counts) / len(correct_mean_counts))
print("wrong max count:", sum(wrong_max_counts) / len(wrong_max_counts))
print("wrong min count:", sum(wrong_min_counts) / len(wrong_min_counts))
print("wrong mean count:", sum(wrong_mean_counts) / len(wrong_mean_counts))
print("total instances:", total_instances)
write_jsonl(analyze_results,mme_output_file)
        



{'numerical_calculation/images/0002.png': ['image', 'piece', 'paper', 'math', 'problem', 'problem', 'answer', 'paper', 'half', 'math', 'problem', 'half', 'answer', 'half', 'writing', 'ink'], 'numerical_calculation/images/0013.png': ['image', 'diagram', 'piece', 'paper', 'graph', 'draw', 'diagram', 'triangle', 'square', 'triangle', 'section', 'triangle', 'corner', 'paper', 'square', 'middle', 'triangle', 'number', 'paper', 'measurement', 'coordinate', 'number', 'triangle', 'square', 'top', 'middle', 'bottom', 'triangle'], 'numerical_calculation/images/0017.png': ['image', 'piece', 'paper', 'equation', 'equation', 'combination', 'number', 'letter', 'number', 'handwriting', 'mix', 'cursive', 'writing', 'equation', 'paper', 'table', 'writing', 'shade', 'pen', 'pencil'], 'numerical_calculation/images/0003.png': ['image', 'equation', 'piece', 'paper', 'number', 'equation', 'style', 'touch', 'paper', 'notebook', 'page', 'writing', 'ink', 'equation', 'focus', 'image', 'object', 'distraction'],

100%|██████████| 2372/2372 [00:00<00:00, 54895.27it/s]

correct max rank: 9738.922376873661
correct min rank: 1.0
correct mean rank: 1035.7382459634152
wrong max rank: 10377.442460317461
wrong min rank: 1.0
wrong mean rank: 1068.2158838121986
correct max count: 625045.0
correct min count: 809.1343683083512
correct mean count: 57531.92400168595
wrong max count: 625045.0
wrong min count: 821.640873015873
wrong mean count: 59439.259703645235
total instances: 326806
